In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!pip install hanja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
%cd "/content/drive/MyDrive/Colab Notebooks/AI_graduation_project/trainer_Retro-Reader"

/content/drive/MyDrive/Colab Notebooks/AI_graduation_project/trainer_Retro-Reader


# 한자 변환, 괄호 띄어쓰기, 글자 이어붙이기

In [22]:
import json
with open('./after_processing_data/Training/training_12m.json', 'r', encoding='utf-8') as f:
  train = json.load(f)
with open('./after_processing_data/Validation/validation_12m.json', 'r', encoding='utf-8') as f:
  dev = json.load(f)

In [23]:
import csv

concat_w = []
with open('./data/concat_words.tsv', 'r', encoding='utf-8') as f:
  tr = csv.reader(f, delimiter='\t')
  for _ in tr:
     concat_w.append(_)

concat_w = sorted(concat_w, key=lambda x: len(x[1]), reverse=True)

In [24]:
# concat_w

In [25]:
import hanja

def blank_and_hanja_and_concat(str1):
  s = hanja.translate(str1, 'substitution')
  s = s.replace("  (", " (").replace(")  ", ") ").replace(":", " :").replace("  :",  " :") \
  .replace("  {", " {").replace("}  ", "} ").replace("  [", " [").replace("]  ", "] ")
  s = s.replace("%uF98E", "연").replace("%uF9B5", "예")
  for c in concat_w:
    s = s.replace(c[0], c[1])
  return s

In [26]:
for data in train['data']:
  p = data['paragraphs'][0]
  p['context'] = blank_and_hanja_and_concat(p['context'])
  for qas in p['qas']:
    if qas['is_impossible'] == False:
      qas['answers'][0]['text'] = blank_and_hanja_and_concat(qas['answers'][0]['text'])
      qas['answers'][0]['answer_start'] = p['context'].find(qas['answers'][0]['text'])

for data in dev['data']:
  p = data['paragraphs'][0]
  p['context'] = blank_and_hanja_and_concat(p['context'])
  for qas in p['qas']:
    if qas['is_impossible'] == False:
      qas['answers'][0]['text'] = blank_and_hanja_and_concat(qas['answers'][0]['text'])
      qas['answers'][0]['answer_start'] = p['context'].find(qas['answers'][0]['text'])

In [27]:
# ./train_10_f.json ./dev_10_f.json
with open('./after_processing_data/Training/training_12m.json', 'w') as f:
  json.dump(train, f, ensure_ascii=False, indent=2)

with open('./after_processing_data/Validation/validation_12m.json', 'w') as f:
  json.dump(dev, f, ensure_ascii=False, indent=2)